# 企业级问答系统 —— 离线知识生产流程

## 一、背景介绍

智能问答系统是当前应用范围最广，最容易落地的大模型应用。通常来说，智能问答系统包括以下五部分：
1. Query理解
2. 混合检索召回
3. 语义排序
4. 答案生成
5. 追问生成

AppBuilder在问答系统沉淀了最佳实践流程，并以极易用的组件化的方式对外开放，可以使用页面操作 + 低代码的方式快速搭建自己的问答系统。

<img src="./qa_system/qa_flow.png" alt="drawing" width="1000"/>


## 二、实操流程

整体使用流程包括以下三个环节：

1. 登录[百度智能云千帆AppBuilder官网](https://cloud.baidu.com/product/AppBuilder)创建账户，

<img src="./qa_system/product.png" alt="drawing" width="1000"/>


2. 在[百度智能云千帆AppBuilder控制台](https://console.bce.baidu.com/ai_apaas/dialogHome)右侧头像栏『API密钥』页面获取密钥，并复制。

<img src="./qa_system/token.jpg" alt="drawing" width="1000"/>


<img src="./qa_system/get_token.png" alt="drawing" width="1000"/>


3. 引用AppBuilder SDK代码，调用Dataset API相关接口，创建数据集，并增、查、删除文档。

### 2.1 环境准备

首先需要安装AppBuilder-SDK代码库，若已在开发环境安装，则可跳过此步。

**注意：**: appbuilder-sdk 的python版本要求 3.9+

In [2]:
!python3 -m pip install appbuilder-sdk

Defaulting to user installation because normal site-packages is not writeable
  Using cached appbuilder_sdk-0.6.0-py3-none-any.whl.metadata (768 bytes)
Using cached appbuilder_sdk-0.6.0-py3-none-any.whl (335 kB)


### 2.2 引入AppBuilder-SDK，并设置TOKEN

**注意：** 请使用刚才在AppBuilder平台上新建的个人TOKEN

In [9]:
# 引入os模块，引入appbuilder 模块
import os
import appbuilder

# 设置appbuilder的token密钥，从页面上复制粘贴token，覆盖此处的 "your_appbuilder_token"
os.environ['APPBUILDER_TOKEN'] = "your_appbuilder_token"

print("AppBuilder 模块导入成功！")
print("您的AppBuilder Token为：{}".format(os.environ['APPBUILDER_TOKEN']))

AppBuilder 模块导入成功！
您的AppBuilder Token为：your_appbuilder_token


### 2.3 创建全新知识库

我们从零开始创建一个新的知识库，并使用它来支持问答系统。我们将其命名为**说明书知识库**

In [4]:
# 创建全新知识库
dataset = appbuilder.console.Dataset.create_dataset("说明书知识库")

print("知识库创建成功！ 知识库的ID为：{}， 知识库的名称为： {}".format(
    dataset.dataset_id,
    dataset.dataset_name))

知识库创建成功！ 知识库的ID为：c368a982-699f-4114-9e9a-c08b76c07f92， 知识库的名称为： 说明书知识库


### 2.4 上传文档到知识库

在notebook的同级目录`./qa_system`下，我们提前准备了两篇文档：
- `./qa_system/平板电脑说明书.pdf`
- `./qa_system/显示器说明书.pdf`

我们将这两篇文档上传到知识库中，用于后续问答应用的私域知识检索。

In [5]:
file_1 = "./qa_system/平板电脑说明书.pdf"
file_2 = "./qa_system/显示器说明书.pdf"

document_infos = dataset.add_documents([
    file_1, file_2
])

print("文档上传到知识库成功！知识库中的文档ID为：{}".format(document_infos.document_ids))


文档上传到知识库成功！知识库中的文档ID为：['c396c715-66d8-49cb-af9d-af0bd08a16e2', '817d29bb-be72-420b-bee1-c1d9b1116826']


### 2.5 查找知识库中的文档

问答系统中的知识库，常用增、删、查操作，我们刚才演示了增加文档的方法，下面演示如何查找文档。

我们获取[百度智能云千帆AppBuilder控制台-我的知识](https://console.bce.baidu.com/ai_apaas/dataset)页面中，我们刚才创建的知识库中的文档列表。

<img src="./qa_system/doc_list.png" alt="drawing" width="1000"/>

In [6]:
doc_list = dataset.get_documents(page=1,limit=10)
print("文档列表获取成功！")
print("第二个的文档是: {}, 文档ID是: {}, 文档字数是: {}".format(
    doc_list.data[1].name, doc_list.data[1].id, doc_list.data[1].word_count))

文档列表获取成功！
第二个的文档是: 平板电脑说明书.pdf, 文档ID是: c396c715-66d8-49cb-af9d-af0bd08a16e2, 文档字数是: 8568


### 2.6 删除知识库中的文档

下面我们演示如何删除知识库中的文档。我们尝试删除 『平板电脑说明书.pdf』 这篇文档

In [7]:
need_delete_doc_name = "平板电脑说明书.pdf"
need_delete_doc_id = ""

doc_list = dataset.get_documents(page=1,limit=10)
for doc in doc_list.data:
    if doc.name == need_delete_doc_name:
        need_delete_doc_id = doc.id
        break

if need_delete_doc_id == "":
    print("未找到指定文档，请检查 2.4 步骤是否正确上传，以及2.5步骤中是否展示了正确的文档列表")
else:
    dataset.delete_documents([need_delete_doc_id])
    print("文档: {}, 文档ID: {} 删除成功!".format(
        need_delete_doc_name,
        need_delete_doc_id
    ))

print("文档删除成功，请刷新『知识库』前端页面，观察是否已无『{}』文档".format(need_delete_doc_name))


文档: 平板电脑说明书.pdf, 文档ID: c396c715-66d8-49cb-af9d-af0bd08a16e2 删除成功!
文档删除成功，请刷新『知识库』前端页面，观察是否已无『平板电脑说明书.pdf』文档


执行完上述步骤后，前端页面刷新，指定文档已经被成功删除

<img src="./qa_system/delete_doc.png" alt="drawing" width="1000"/>

In [8]:
# 再一次确认当前知识库的 Name 与 ID，方便后续 企业级问答系统 —— 在线问答流程 的使用

print("当前创建的知识库的ID为：{}， 知识库的名称为： {}".format(
    dataset.dataset_id,
    dataset.dataset_name))

当前创建的知识库的ID为：c368a982-699f-4114-9e9a-c08b76c07f92， 知识库的名称为： 说明书知识库


## 三、总结

AppBuilder 为 基于RAG的 企业问答系统提供了 `appbuilder.console.dataset` API，可以方便的以代码态进行知识库的管理，实现基本的增删改查业务需求。

本示例展示了如何从零创建知识库，并实现增、查、改的代码，并可以观察SDK与前端页面的联动。